In [ ]:
import numpy as np
import pandas as pd
from datetime import *
import tensorflow as tf
from tensorflow.keras import regularizers
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Dropout, Input, Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose, TimeDistributed
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from pathlib import Path
import csv


In [ ]:
# GPU memory configuration
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [ ]:
# Load experimental data
exp_0_deg = [
    0.849617672047573, 0,
    10.4078164825828, -0.0357142857142858,
    25.2761257434155, -0.0779220779220779,
    50.339847068819, -0.194805194805195,
    100.254885301614, -0.470779220779221,
    152.293967714528, -0.876623376623377
]

exp_0_deg_q = exp_0_deg[0::2]
exp_0_deg_theta = exp_0_deg[1::2]

exp_1_15_deg = [
    50.339847068819, -0.0714285714285714,
    169.498725573492, -0.321428571428571,
    204.333050127443, -0.457792207792208
]

exp_1_15_deg_q = exp_1_15_deg[0::2]
exp_1_15_deg_theta = exp_1_15_deg[1::2]

exp_3_15_deg = [
    0.849617672047573, 0.00649350649350655,
    10.4078164825828, 0.0324675324675323,
    50.339847068819, 0.220779220779221,
    100.254885301614, 0.487012987012987,
     115.335598980459, 0.594155844155844,
    135.301614273577, 0.75,
    152.293967714528, 0.902597402597403,
    169.28632115548, 1.07467532467532
]

exp_3_15_deg_q = exp_3_15_deg[0::2]
exp_3_15_deg_theta = exp_3_15_deg[1::2]

exp_4_deg = [
    50.1274426508071, 0.327922077922078,
    134.876805437553, 1.22727272727273,
    169.073916737468, 1.62012987012987,
    203.908241291419, 2.2012987012987
]

exp_4_deg_q = exp_4_deg[0::2]
exp_4_deg_theta = exp_4_deg[1::2]

exp_5_deg = [
    0.637213254035684, 0.00649350649350655,
    24.6389124893798, 0.214285714285714,
    49.9150382327952, 0.464285714285714,
    74.553950722175, 0.733766233766234,
    100.042480883602, 1.04545454545455,
    120.00849617672, 1.33116883116883,
    135.514018691589,1.54911499194776,
    152.081563296517,1.8022657092623,
    169.28632115548,2.0651529926274
]

exp_5_deg_q = exp_5_deg[0::2]
exp_5_deg_theta = exp_5_deg[1::2]

exp_6_5_deg = [
    0.637213254035684, 0,
    10.4078164825828, 0.0844155844155845,
    25.0637213254036, 0.217532467532468,
    50.1274426508071, 0.37987012987013,
    100.254885301614, 0.727272727272727,
    135.089209855565, 0.996753246753247,
    152.293967714528, 1.12662337662338,
    169.073916737468, 1.2987012987013
]

exp_6_5_deg_q = exp_6_5_deg[0::2]
exp_6_5_deg_theta = exp_6_5_deg[1::2]



In [ ]:
model_dir = 'Model_architecture\\FCNN_v1\\'

# Inputs for aerodynamic forces estimation
xref = 0.12192          # Reference point for moment calculation (m)
chord = 0.4064          # Chord length of the wing (m)
x_30 = 0.12192          # 30% chord location (m)
x_50 = 0.2032           # 50% chord location (m)
span = chord*2          # Span of the wing (m)
area = span*chord       # Reference area (m^2)


# Create directories for the results
directory_results = 'Results\\Static_deformation\\'
Path(directory_results).mkdir(parents=True, exist_ok=True)


# Load dataset
dataset = np.load('Dataset_NN\\dataset.npy')
grid_data = 'Dataset\\grid_data.dat'
CFD_csv_file = 'Dataset\\surface.csv' # File with normals and cell area

# print( '\n-> Shape of the loaded matrix: \n')
# print(dataset.shape) # x, y , z , CP , CF_x, CF_y, CF_z, M , AoA

X = dataset[:,:,[0,1,2,7,8]] # Input
Y = dataset[:,:,3:7]         # Output


## Feature normalisation:
scaler = MinMaxScaler(feature_range=(-1, 1))
samples, points, variables = X.shape
X = np.reshape(X, newshape=(-1, variables))
X = scaler.fit_transform(X)
X = np.reshape(X, newshape=(samples, points, variables))
#print(X.shape)
scalery = MinMaxScaler(feature_range=(-1, 1))
# Y = np.expand_dims(Y,axis=2)
samples, points, variables = Y.shape
Y = np.reshape(Y, newshape=(-1, variables))
Y = scalery.fit_transform(Y)
Y = np.reshape(Y, newshape=(samples, points, variables))

In [ ]:
model = tf.keras.models.load_model(model_dir+ 'neural_network_model.h5')
model.load_weights(model_dir+ 'neural_network_weights.h5')
print(model.summary())

# Denormalize
samples, points, variables = X.shape
X_test = np.reshape(X, newshape=(-1, variables))
X_test = scaler.inverse_transform(X_test)
X_test = np.reshape(X_test, newshape=(samples, points, variables))

cooordinates = pd.read_csv(CFD_csv_file)

In [ ]:
def neural_network(Mach, AoA, npts, X_temp):
  """
  Predicts aerodynamic lift and moment using a trained neural network model.

  Parameters:
    Mach (float): Mach number for prediction.
    AoA (float): Angle of attack in degrees.
    npts (int): Number of points in the input grid.
    X_temp (np.ndarray): Input grid coordinates (npts x 3).

  Returns:
    tuple: (lift_NN, moment_NN) - Predicted lift and moment.
  """
  # Create arrays for Mach and AoA for each point
  Mach_i = np.full((npts, 1), Mach)  # Mach column
  AoA_i = np.full((npts, 1), AoA)    # AoA column

  # Concatenate coordinates with Mach and AoA
  X_exp = np.concatenate((X_temp, Mach_i, AoA_i), axis=1)  # Shape: (npts, 5)

  # Normalize input features
  X_exp = scaler.transform(X_exp)
  X_exp = np.expand_dims(X_exp, axis=0)  # Add batch dimension

  # Predict using the neural network model
  Y_exp_pred = model.predict(X_exp, batch_size=1, verbose=0)

  # Denormalize input for further processing
  samples, points, variables = X_exp.shape
  X_exp = np.reshape(X_exp, newshape=(-1, variables))
  X_exp = scaler.inverse_transform(X_exp)
  X_exp = np.reshape(X_exp, newshape=(samples, points, variables))

  # Denormalize output predictions
  samples, points, variables = Y_exp_pred.shape
  Y_exp_pred = np.reshape(Y_exp_pred, newshape=(-1, variables))
  Y_exp_pred = scalery.inverse_transform(Y_exp_pred)
  Y_exp_pred = np.reshape(Y_exp_pred, newshape=(samples, points, variables))

  # Read grid data to get cell areas for weighting
  df = pd.read_csv(grid_data, sep=' ')
  cell_area = np.array(df.Cell_Volume)
  mean_area = np.mean(cell_area)
  cell_area = cell_area / mean_area  # Normalize cell areas

  # Combine input and predicted output into a DataFrame
  df_combined = np.concatenate((X_exp[0, :, :], Y_exp_pred[0, :, :]), axis=1)
  df_combined = pd.DataFrame(df_combined, columns=[
    'x', 'y', 'z', 'Mach', 'AoA',
    'Pressure_Coefficient', 'Skin_Friction_Coefficient_x',
    'Skin_Friction_Coefficient_y', 'Skin_Friction_Coefficient_z'
  ])

  # Extract predicted coefficients
  pressure_dataset = df_combined.Pressure_Coefficient
  cf_x_dataset = df_combined.Skin_Friction_Coefficient_x
  cf_y_dataset = df_combined.Skin_Friction_Coefficient_y
  cf_z_dataset = df_combined.Skin_Friction_Coefficient_z

  # Prepare dataset dictionary for further calculations
  dataset = {
    'PointID': cooordinates.PointID,
    'x': cooordinates.x,
    'y': cooordinates.y,
    'z': cooordinates.z,
    'Pressure_Coefficient': pressure_dataset,
    'Skin_Friction_Coefficient_x': cf_x_dataset,
    'Skin_Friction_Coefficient_y': cf_y_dataset,
    'Skin_Friction_Coefficient_z': cf_z_dataset
  }

  # Convert AoA to radians for trigonometric calculations
  aoa = AoA * np.pi / 180

  # Read grid data again for normal vectors and cell areas
  df_cfd = pd.read_csv(grid_data, sep=' ')
  nn_data = pd.DataFrame(data=dataset)

  def compute_aero_coeff(ds):
    """
    Computes aerodynamic lift, drag, and moment coefficients.

    Parameters:
      ds (pd.DataFrame): DataFrame containing surface data and predicted coefficients.

    Returns:
      tuple: (lift, drag, moment)
    """
    coordinates = [ds.x, ds.y, ds.z]  # Surface coordinates
    shear = [
      ds.Skin_Friction_Coefficient_x,
      ds.Skin_Friction_Coefficient_y,
      ds.Skin_Friction_Coefficient_z
    ]  # Shear coefficients
    cp = ds.Pressure_Coefficient  # Pressure coefficient
    normal = [
      df_cfd.X_Grid_K_Unit_Normal,
      df_cfd.Y_Grid_K_Unit_Normal,
      df_cfd.Z_Grid_K_Unit_Normal
    ]  # Surface normals
    cell_area = df_cfd.Cell_Volume  # Cell areas

    # Calculate force components per area
    taux = (shear[0] - normal[0] * cp) / area
    tauz = (shear[2] - normal[2] * cp) / area

    # Integrate force components over the surface
    fz = np.sum(tauz * cell_area)
    fx = np.sum(taux * cell_area)

    # Calculate lift and drag using AoA
    lift = fz * np.cos(aoa) - fx * np.sin(aoa)
    drag = fx * np.cos(aoa) + fz * np.sin(aoa)

    # Calculate aerodynamic moment about reference point
    my = (
      coordinates[2] * (taux * np.cos(aoa) + tauz * np.sin(aoa))
      - (coordinates[0] - xref) * (tauz * np.cos(aoa) - taux * np.sin(aoa))
    ) / chord
    moment = np.sum(my * cell_area)
    return lift, drag, moment

  # Compute aerodynamic coefficients from predicted data
  lift_NN, drag_NN, moment_NN = compute_aero_coeff(nn_data)

  return lift_NN, moment_NN  # Return lift and moment predictions

In [ ]:
########################################
### PREDICTION FOR STATIC DEFORMATION
########################################

print("\n")
print('Prediction for static aeroelastic twist angle')
print("\n")

Mach = 0.74  # Mach number
alpha_range = [0.0, 1.15, 3.15, 4.0, 5.0]  # Angle of attack values (deg)

# 2DOF SS system parameters
mass = 87.91  # kg
Iy = 3.765    # kg m^2
kz = mass * (3.33 * 2 * np.pi) ** 2  # Plunge stiffness (N/m)
ktheta = Iy * (5.20 * 2 * np.pi) ** 2  # Pitch stiffness (Nm/rad)
V = Mach * np.sqrt(1.116 * 81.49 * 304.2128)  # Flight velocity (m/s)

theta_values_range = []  # To store theta values for each AoA
dynamic_pressure_range = []  # To store dynamic pressure for each AoA

# Define the stiffness matrix for the system
K_aa = np.array([[ktheta, 0], [0, kz]])

for alpha0 in alpha_range:
  print('AoA = ' + str(alpha0))  # Print current angle of attack
  theta_values = []  # Store theta for this AoA
  dynamic_pressure = []  # Store dynamic pressure for this AoA

  for rho_value in np.arange(0.0, 1.3, 0.1):  # Loop over density ratios
    rho = 1.1751 * rho_value  # Air density (kg/m^3)
    q = 0.5 * rho * V * V  # Dynamic pressure (N/m^2)
    print('Dynamic pressure = ' + str(int(q / 47.88)))  # Print dynamic pressure in psf
    input_vector_old = np.array([0, 0])  # Initial guess for [theta, plunge]
    alpha_temp = alpha0  # Temporary AoA for iteration

    for i in range(1000):  # Iterative solver for static equilibrium
      # Predict CL and CM using the neural network
      CL_pred, CM_pred_temp = neural_network(Mach, alpha_temp, npts=X.shape[1], X_temp=X_test[0, :, :3])
      CM_pred = CM_pred_temp + (x_50 - x_30) * CL_pred / chord  # Adjust CM for reference point

      lift_pred = CL_pred * q * area  # Predicted lift (N)
      moment_pred = CM_pred * q * area * chord  # Predicted moment (Nm)

      output = np.array([moment_pred, lift_pred])  # Output vector [moment, lift]

      inv_K_aa = np.linalg.inv(K_aa)  # Inverse of stiffness matrix
      beta = 0.5  # Relaxation factor for convergence
      input_vector = np.dot(inv_K_aa, output) * beta + input_vector_old * (1 - beta)  # Update input vector

      theta = input_vector[0]  # Extract theta (twist angle)
      plunge = input_vector[1]  # Extract plunge (vertical displacement)

      alpha_temp = alpha0 + np.rad2deg(theta)  # Update AoA with twist

      # Check for convergence in theta
      if np.abs(theta - input_vector_old[0]) < 1e-6:
        break

      input_vector_old = input_vector  # Update for next iteration

    theta_values.append(theta)  # Store converged theta
    dynamic_pressure.append(q)  # Store corresponding dynamic pressure

  theta_values_range.append(theta_values)  # Store all theta for this AoA
  dynamic_pressure_range.append(dynamic_pressure)  # Store all q for this AoA

# Convert results to numpy arrays and degrees
theta_values_range = np.array(np.rad2deg(theta_values_range))  # Convert theta to degrees
dynamic_pressure_range = np.array(dynamic_pressure_range)  # Dynamic pressure array

# Create column names for CSV
columns = ['q'] + [f'theta_{alpha}_deg' for alpha in alpha_range]

# Combine dynamic pressure and theta values for CSV
data = np.hstack((np.expand_dims(dynamic_pressure_range[0, :], axis=1), theta_values_range.T))

# Save results to CSV file
filename = 'Static_aeroelastic_twist_angles.csv'
with open(filename, 'w', newline='') as file:
  writer = csv.writer(file)
  writer.writerow(columns)  # Write header
  writer.writerows(data)    # Write data rows

# Plotting
colors = ['cyan', 'pink', 'orange', 'green', 'grey', 'yellow']  # Colors for each AoA
circle_size = 80  # Size of scatter points
linewidth_ = 3  # Line width for plots
plt.rcParams.update({'font.size': 24})  # Set font size for plot
fig, ax = plt.subplots(1, 1)  # Create figure and axis
fig.set_size_inches(18.5, 10.5)  # Set figure size

for i in range(len(alpha_range)):
  ax.plot(dynamic_pressure_range[i, :] / 47.88, theta_values_range[i, :], linewidth=linewidth_,
      color=colors[i], label='AoA = ' + str(alpha_range[i]))  # Plot theta vs q for each AoA

# Plot experimental data as scatter points
ax.scatter(exp_0_deg_q, exp_0_deg_theta, color=colors[0], edgecolors='black', s=circle_size)
ax.scatter(exp_1_15_deg_q, exp_1_15_deg_theta, color=colors[1], edgecolors='black', s=circle_size)
ax.scatter(exp_3_15_deg_q, exp_3_15_deg_theta, color=colors[2], edgecolors='black', s=circle_size)
ax.scatter(exp_4_deg_q, exp_4_deg_theta, color=colors[3], edgecolors='black', s=circle_size)
ax.scatter(exp_5_deg_q, exp_5_deg_theta, color=colors[4], edgecolors='black', s=circle_size)

ax.set_xlabel(r'$Dynamic \,\, Pressure \,\, [psf]$')  # X-axis label
ax.set_ylabel(r'$\theta \,\, [deg]$')  # Y-axis label
ax.set_xlim(0, 250)  # Set x-axis limits
ax.grid()  # Show grid
ax.legend(loc='upper left', fontsize=22)  # Show legend
plt.show()  # Display plot
